<a href="https://colab.research.google.com/github/Arayandra/Andraaseventeen/blob/main/Tugas_Sistem_Cerdas_Pertemuan_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import library
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from sklearn.model_selection import train_test_split

In [14]:
# Load dataset
# Memuat data pengguna, film, dan rating dari dataset MovieLens
# Melakukan preprocessing data seperti normalisasi, pembagian data latih dan uji, dll

links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
tags = pd.read_csv('tags.csv')
ratings = pd.read_csv('ratings.csv')

# Merge movies and ratings data
movies_ratings = pd.merge(ratings, movies, on='movieId')

# Mapping user and item IDs to sequential integers
user_mapping = {user_id: idx for idx, user_id in enumerate(ratings['userId'].unique())}
item_mapping = {item_id: idx for idx, item_id in enumerate(ratings['movieId'].unique())}

num_users = len(user_mapping)
num_items = len(item_mapping)

ratings['user'] = ratings['userId'].map(user_mapping)
ratings['item'] = ratings['movieId'].map(item_mapping)

train, test = train_test_split(ratings, test_size=0.2, random_state=42)

train_user = np.array(train['user'])
train_item = np.array(train['item'])
train_ratings = np.array(train['rating'])

test_user = np.array(test['user'])
test_item = np.array(test['item'])
test_ratings = np.array(test['rating'])


# Bagian Baru

In [6]:
# Create model
# Membangun model NCF menggunakan TensorFlow/Keras
def create_ncf_model(num_users, num_items, latent_dim=8):
  # Input layer untuk user
  user_input = Input(shape=(1,))
  user_embedding = Embedding(num_users, latent_dim)(user_input)
  user_flat = Flatten()(user_embedding)
  # Input layer untuk item (film)
  item_input = Input(shape=(1,))
  item_embedding = Embedding(num_items, latent_dim)(item_input)
  item_flat = Flatten()(item_embedding)
  # Concatenate user dan item embeddings
  concat = Concatenate()([user_flat, item_flat])
  # Fully connected layer
  hidden = Dense(64, activation='relu')(concat)
  output = Dense(1)(hidden)
  # Compile model
  model = Model(inputs=[user_input, item_input], outputs=output)
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model


In [7]:
# Train model
# Melatih model menggunakan data latih
model = create_ncf_model(num_users, num_items)
model.fit([train_user, train_item], train_ratings, batch_size=64, epochs=10,
validation_data=([test_user, test_item], test_ratings))

Epoch 1/10
1261/1261 [==============================] - 5s 3ms/step - loss: 1.6172 - val_loss: 0.7877
Epoch 2/10
1261/1261 [==============================] - 4s 3ms/step - loss: 0.7166 - val_loss: 0.7803
Epoch 3/10
1261/1261 [==============================] - 5s 4ms/step - loss: 0.6817 - val_loss: 0.7780
Epoch 4/10
1261/1261 [==============================] - 4s 3ms/step - loss: 0.6581 - val_loss: 0.7600
Epoch 5/10
1261/1261 [==============================] - 4s 3ms/step - loss: 0.6411 - val_loss: 0.7546
Epoch 6/10
1261/1261 [==============================] - 5s 4ms/step - loss: 0.6282 - val_loss: 0.7576
Epoch 7/10
1261/1261 [==============================] - 3s 3ms/step - loss: 0.6161 - val_loss: 0.7576
Epoch 8/10
1261/1261 [==============================] - 4s 3ms/step - loss: 0.6035 - val_loss: 0.7579
Epoch 9/10
1261/1261 [==============================] - 4s 3ms/step - loss: 0.5912 - val_loss: 0.7585
Epoch 10/10
1261/1261 [==============================] - 5s 4ms/step - loss: 0.578

In [8]:
# Evaluate model
# Evaluasi model menggunakan data uji
test_loss = model.evaluate([test_user, test_item], test_ratings)
print(f"Test Loss: {test_loss}")

631/631 [==============================] - 1s 2ms/step - loss: 0.7639
Test Loss: 0.7639102935791016


In [12]:
# Make recommendations
# Menggunakan model untuk membuat rekomendasi untuk pengguna tertentu
def movies_not_rated_by_user(user_id):
    user_rated_movies = set(ratings[ratings['user'] == user_mapping[user_id]]['item'])
    all_movies = set(range(num_items))
    not_rated_movies = all_movies - user_rated_movies
    return list(not_rated_movies)

def get_movie_titles(movie_ids):
    movie_titles = movies[movies['movieId'].isin(movie_ids)]['title'].tolist()
    return movie_titles

user_id = 123
user_movies = movies_not_rated_by_user(user_id)
user_input = np.array([user_id] * len(user_movies))
user_movies_input = np.array(user_movies)
predictions = model.predict([user_input, user_movies_input]).flatten()
recommended_movie_indices = np.argsort(predictions)[::-1][:10]
recommended_movie_ids = [user_movies[idx] for idx in recommended_movie_indices]

303/303 [==============================] - 1s 1ms/step


In [13]:
# Display recommended movies to the user
recommended_movies = get_movie_titles(recommended_movie_ids)
print("Recommended movies for user", user_id, ":", recommended_movies)

Recommended movies for user 123 : ["Logan's Run (1976)", 'American Werewolf in Paris, An (1997)', 'Sweet Sixteen (2002)']
